<a href="https://colab.research.google.com/github/pratik-poudel/datacrunch/blob/main/june20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
import numpy as np
import pandas as pd
from collections import defaultdict
from tqdm.notebook import tqdm
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns
import gc
# pd.options.display.float_format| = "{:.2f}".format
pd.options.display.max_columns = 500

import requests
from scipy import stats

import lightgbm as lgb

# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

import sklearn
from sklearn import *
from sklearn.experimental import enable_hist_gradient_boosting
# from sklearn.utils.testing import all_estimators
from sklearn.base import RegressorMixin
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report 
from sklearn.metrics import make_scorer
from sklearn.model_selection import RepeatedKFold, cross_val_score, KFold, GroupKFold
from sklearn.multioutput import RegressorChain


import requests
import time
from scipy import stats

import lightgbm as lgb
# import xgboost as xgb

import matplotlib.pyplot as plt
import matplotlib.style as style
import seaborn as sns
from matplotlib import pyplot
from matplotlib.ticker import ScalarFormatter
sns.set_context("talk")
style.use('seaborn-colorblind')


from itertools import combinations, permutations, combinations_with_replacement


In [36]:
train_datalink_X = 'https://tournament.datacrunch.com/data/X_train.csv'  
train_datalink_y = 'https://tournament.datacrunch.com/data/y_train.csv'
hackathon_data_link = 'https://tournament.datacrunch.com/data/X_test.csv'

train = pd.read_csv(train_datalink_X) # X
target = pd.read_csv(train_datalink_y) # y
test = pd.read_csv(hackathon_data_link)

train.columns = train.columns.str.lower()
target.columns = target.columns.str.lower()
test.columns = test.columns.str.lower()

train.drop(columns=['id'], inplace=True)
test.drop(columns=['id'], inplace=True)

In [42]:
train.shape, test.shape

((624650, 50), (3746, 50))

In [43]:
train_all = pd.concat([train, target], axis=1)

In [45]:
moons = [557, 558]
test = train_all[train_all.moons.isin(moons)]

In [47]:
test_target = test[['target_r', 'target_g', 'target_b']]
test = test.drop(columns = ['target_r', 'target_g', 'target_b'] )

In [50]:
train_all.drop(test.index, inplace=True)

In [51]:
train = train_all.drop(columns = ['target_r', 'target_g', 'target_b'] )
target = train_all[['target_r', 'target_g', 'target_b']]

In [54]:
r = train.corrwith(target.target_r, method='spearman')
g = train.corrwith(target.target_g, method='spearman')
b = train.corrwith(target.target_b, method='spearman')

In [56]:
r.sort_values()

feature_32   -0.023660
feature_30   -0.023222
feature_31   -0.022422
feature_29   -0.022245
feature_34   -0.021979
feature_33   -0.021973
feature_35   -0.021797
feature_14   -0.020754
feature_13   -0.020295
feature_17   -0.019603
feature_18   -0.018544
feature_20   -0.017796
feature_15   -0.017727
feature_11   -0.017515
feature_8    -0.017375
feature_10   -0.017126
feature_21   -0.016747
feature_19   -0.016445
feature_12   -0.016340
feature_9    -0.015862
feature_1    -0.015097
feature_16   -0.014959
feature_36   -0.014229
feature_37   -0.012348
feature_38   -0.011707
feature_39   -0.011148
feature_40   -0.010997
feature_22   -0.010861
feature_41   -0.010825
feature_42   -0.010689
feature_2    -0.010390
feature_43   -0.010322
feature_44   -0.007763
feature_23   -0.007716
feature_3    -0.007307
feature_24   -0.006233
feature_45   -0.006015
feature_4    -0.005425
feature_46   -0.005349
feature_5    -0.003636
feature_47   -0.003299
feature_25   -0.002683
feature_48   -0.001835
feature_49 

In [57]:
def calculate_combination_corr(train_dataset, train_target, combination):

    features = [f for f in train_dataset.columns if 'feature' in f]
    X = train_dataset[features].copy()
    y = train_target.copy()
    
    comb= []
    for i in combinations(X.columns, combination):
        comb.append(i)
    score_r = []
    score_g = []
    score_b = []
    
    for i in tqdm(comb):
        tr = X[list(i)].values
        cir_mean = stats.circmean(tr, axis=1)
        spearman_r  = stats.spearmanr(cir_mean, y.target_r.values)[0]
        spearman_g  = stats.spearmanr(cir_mean, y.target_g.values)[0]
        spearman_b  = stats.spearmanr(cir_mean, y.target_b.values)[0]
    
        score_r.append(spearman_r)
        score_g.append(spearman_g)
        score_b.append(spearman_b)

    return pd.DataFrame(zip(comb, score_r, score_g, score_b), columns=['feature', 'score_r', 'score_g', 'score_b'])

In [65]:
def run_lgb(train_dataset, target, best_fold, actual_test):
    num_boost_round= 1000
    params = {
        'boosting_type': 'gbdt',
        'objective': 'regression',
        'metric': 'rmse',
        'random_state':60,
    }
    X =  train_dataset.copy()
    y = target[['target_r', 'target_g', 'target_b']]

    oof_pred_r = np.zeros(len(X))
    oof_pred_g = np.zeros(len(X))
    oof_pred_b = np.zeros(len(X))

    original_test_preds_r = np.zeros(len(test))
    original_test_preds_g = np.zeros(len(test))
    original_test_preds_b = np.zeros(len(test))

    kf = GroupKFold(n_splits=best_fold)

    features = [f for f in X.columns if 'feature' in f]
    cnt = 1
    for tr_ind , val_ind in kf.split(X, y, 
                                    X['moons']
                                    ):
        print(f'Fold:{cnt}, Train set: {len(tr_ind)}, Test set:{len(val_ind)}') 
        X= X[features]
        X_train, X_val = X.iloc[tr_ind], X.iloc[val_ind]
        y_train, y_val = y.iloc[tr_ind], y.iloc[val_ind]
      
        # print(X_train.shape)
        tr_data_r = lgb.Dataset(X_train, y_train.target_r)
        va_data_r = lgb.Dataset(X_val, y_val.target_r)

        tr_data_g = lgb.Dataset(X_train, y_train.target_g)
        va_data_g = lgb.Dataset(X_val, y_val.target_g)

        tr_data_b = lgb.Dataset(X_train, y_train.target_b)
        va_data_b = lgb.Dataset(X_val, y_val.target_b)
        
        evals_result = {}

        print("\nMODEL R *********************************************************************************************")
        model_r = lgb.train(params, tr_data_r, 
                            valid_sets=[tr_data_r, va_data_r], 
                            verbose_eval=50, 
                            num_boost_round=num_boost_round, 
                            early_stopping_rounds=50,
                            # feval=spearman_r, 
                            evals_result=evals_result)
        print("\nMODEL G *********************************************************************************************")

        model_g = lgb.train(params, tr_data_g, 
                            valid_sets=[tr_data_g, va_data_g], 
                            verbose_eval=50, 
                            num_boost_round=num_boost_round, 
                            early_stopping_rounds=50,
                            # feval = spearman_g, 
                            evals_result=evals_result)
        
        print("\nMODEL B *********************************************************************************************")
        model_b = lgb.train(params, tr_data_b, 
                            valid_sets=[tr_data_b, va_data_b], 
                            verbose_eval=50, 
                            num_boost_round=num_boost_round, 
                            early_stopping_rounds=50, 
                            # feval = spearman_b, 
                            evals_result=evals_result)

        val_pred_r = model_r.predict(X_val, num_iteration= model_r.best_iteration)
        val_pred_g = model_g.predict(X_val,  num_iteration= model_r.best_iteration)
        val_pred_b = model_b.predict(X_val,  num_iteration= model_r.best_iteration)

        oof_pred_r[val_ind] = val_pred_r
        oof_pred_g[val_ind] = val_pred_g
        oof_pred_b[val_ind] = val_pred_b

        print("FOLD ", cnt, "COMPLETED")
        # print()
        # print("*******" * 10  )
        cnt += 1
        
        actual_test = actual_test[features]
        print(actual_test.shape)
        original_test_preds_r += model_r.predict(actual_test,  num_iteration= model_r.best_iteration)
        original_test_preds_g += model_g.predict(actual_test,  num_iteration= model_r.best_iteration)
        original_test_preds_b += model_b.predict(actual_test,  num_iteration= model_r.best_iteration)

    original_test_preds_r = original_test_preds_r / kf.n_splits
    original_test_preds_g = original_test_preds_g / kf.n_splits
    original_test_preds_b = original_test_preds_b / kf.n_splits


    oof_pred_df = pd.concat([pd.DataFrame(oof_pred_r, columns=['target_r']),
                            pd.DataFrame(oof_pred_g, columns=['target_g']),
                            pd.DataFrame(oof_pred_b, columns=['target_b'])], axis=1)

    score_r = stats.spearmanr(oof_pred_df.target_r, y.target_r)[0] * 100
    score_g = stats.spearmanr(oof_pred_df.target_g, y.target_g)[0] * 100
    score_b = stats.spearmanr(oof_pred_df.target_b, y.target_b)[0] * 100


    prediction = pd.concat([pd.DataFrame(original_test_preds_r, columns=['target_r']),
                    pd.DataFrame(original_test_preds_g, columns=['target_g']),
                    pd.DataFrame(original_test_preds_b, columns=['target_b'])], axis=1)
    print(score_r, score_g, score_b)
    score = (score_r+score_g+score_b)/ 3
    print(score)

    return oof_pred_df, prediction, model_r, model_g, model_b

In [66]:
oof_pred_df, prediction, r, g, b = run_lgb(train, target,5, test)

Fold:1, Train set: 496245, Test set:124659

MODEL R *********************************************************************************************
Training until validation scores don't improve for 50 rounds.
[50]	training's rmse: 0.352063	valid_1's rmse: 0.352528
[100]	training's rmse: 0.350735	valid_1's rmse: 0.351732
[150]	training's rmse: 0.349513	valid_1's rmse: 0.351033
[200]	training's rmse: 0.348337	valid_1's rmse: 0.350358
[250]	training's rmse: 0.347162	valid_1's rmse: 0.349685
[300]	training's rmse: 0.346075	valid_1's rmse: 0.34908
[350]	training's rmse: 0.345057	valid_1's rmse: 0.348529
[400]	training's rmse: 0.344017	valid_1's rmse: 0.34798
[450]	training's rmse: 0.343029	valid_1's rmse: 0.347462
[500]	training's rmse: 0.342093	valid_1's rmse: 0.347031
[550]	training's rmse: 0.341145	valid_1's rmse: 0.346565
[600]	training's rmse: 0.340181	valid_1's rmse: 0.346082
[650]	training's rmse: 0.339235	valid_1's rmse: 0.345593
[700]	training's rmse: 0.338372	valid_1's rmse: 0.3451

In [68]:
prediction.shape

(3746, 3)

In [69]:
test_target.shape

(3746, 3)

In [72]:
print(stats.spearmanr(prediction.target_r, test_target.target_r)[0])
print(stats.spearmanr(prediction.target_g, test_target.target_g)[0])
print(stats.spearmanr(prediction.target_b, test_target.target_b)[0])


0.05979943930853575
0.2595891709920574
0.3194896525405518


In [79]:
feat = [f for f in train.columns if 'f' in f]

In [85]:
r1_features = pd.Series(r.feature_importance(importance_type='gain', iteration=r.best_iteration), index=train[feat].columns).sort_values(ascending=False)
g1_features= pd.Series(g.feature_importance(importance_type='gain', iteration=g.best_iteration), index=train[feat].columns).sort_values(ascending=False)
b1_features= pd.Series(b.feature_importance(importance_type='gain', iteration=b.best_iteration), index=train[feat].columns).sort_values(ascending=False)

In [97]:
features= list(set(r1_features[:10].index.to_list() + g1_features[:10].index.to_list() + b1_features[:10].index.to_list()))

In [98]:
features

['feature_14',
 'feature_11',
 'feature_16',
 'feature_19',
 'feature_18',
 'feature_13',
 'feature_8',
 'feature_15',
 'feature_9',
 'feature_12',
 'feature_47',
 'feature_17',
 'feature_10']

In [99]:
train = train[features]
test= test[features]

In [100]:
df2 = calculate_combination_corr(train, target, 2)

In [101]:
df3 = calculate_combination_corr(train, target, 3)
df4 = calculate_combination_corr(train, target, 4)

In [102]:
df = pd.concat([df2, df3, df4])
df

,feature,score_r,score_g,score_b
0,"(feature_14, feature_11)",-0.022283,-0.036843,-0.049434
1,"(feature_14, feature_16)",-0.020565,-0.035444,-0.047697
2,"(feature_14, feature_19)",-0.021508,-0.037161,-0.049582
3,"(feature_14, feature_18)",-0.023015,-0.039380,-0.051769
4,"(feature_14, feature_13)",-0.023947,-0.039256,-0.052328
...,...,...,...,...
710,"(feature_9, feature_12, feature_47, feature_17)",-0.021438,-0.036943,-0.050700
711,"(feature_9, feature_12, feature_47, feature_10)",-0.020210,-0.035505,-0.049120
712,"(feature_9, feature_12, feature_17, feature_10)",-0.022310,-0.038760,-0.053454
713,"(feature_9, feature_47, feature_17, feature_10)",-0.021243,-0.037135,-0.050845


In [121]:
df.score_r = df.score_r.abs()
df.score_g = df.score_b.abs()
df.score_b = df.score_b.abs()


In [123]:
df.max()

feature    feature_9
score_r    0.0252677
score_g    0.0572931
score_b    0.0572931
dtype: object

In [127]:
sr = train.corrwith(target.target_r, method='spearman')
sg = train.corrwith(target.target_g, method='spearman')
sb = train.corrwith(target.target_b, method='spearman')

In [126]:
sr.abs().max(), sg.abs().max(), sb.abs().max()

(0.020753757562699135, 0.03439893954224204, 0.045312532274375074)